In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("data_set_utf8.csv")

In [ ]:
df.dtypes

In [ ]:
df.shape

In [ ]:
print(df.isna().sum().sort_values(ascending=False))
# print(df.isna().sum())

In [36]:
def missing(dff):
    print (round((dff.isnull().sum() * 100/ len(dff)),2).sort_values(ascending=False))

In [ ]:
df.head(30)

In [3]:
# Wyświetlanie całego wiersza 
pd.set_option('display.max_colwidth', -1)

#### Id

In [ ]:
# ile pustych
len(df[df.id.isna()])
# ile unikalnych wartości
len(df.id.unique())
# id
df.id.head()

<b>offer-item-ad_id48H25</b> <--- wygląd pojedyńczej wartości
<br>
<b>id48H25</b> <--- wygląd docelowy

In [4]:
df['id'] = df['id'].apply(lambda x: x.split('_')[1])

In [ ]:
df.id.head()

#### district

In [ ]:
df.district.head()

<b>Mieszkanie na sprzedaż: Wrocław, Fabryczna, Grabiszynek</b> <--- tak wygląda surowa wartość
<br>
<b>Fabryczna</b> <--- docelowa wartość
<br>
Dodatkowo pojawia się wartości dzielnicy: 
<ul>
    <li><b>dolnośląskie</b> - w przypadku takiej wartości wstawiam <i>np.NaN</i></li>
    <li><b>Rynek</b> - w przypadku takiej wartości wstawiam <i>Stare Miasto</i></li>
    <li><b>Nadodrze</b> - w przypadku takiej wartości wstawiam <i>Śródmieście</i></li> 
</ul>

In [5]:
df['district'] = df['district'].apply(lambda x: np.NaN if x.split(',')[1].strip() == 'dolnośląskie' 
                                      else x.split(',')[1].strip().replace("Rynek","Stare Miasto").replace("Nadodrze","Śródmieście")) 

In [ ]:
df.district.unique()

In [ ]:
df.district.head()

#### Price

In [ ]:
df.price.head()

<b>590 000 zł</b> <--- tak wygląda surowa wartość
<br>
<b>590000.0</b> <--- docelowa wartość
<br>
Dodatkowo pojawia się wartości ceny: 
<ul>
    <li><b>Zapytaj o cenę</b> - w przypadku takiej wartości wstawiam <i>np.NaN</i></li>
</ul>

In [6]:
df['price'] = df['price'].apply(lambda x: pd.to_numeric(x.split(' z')[0].strip().replace(' ', '').replace(',','.')) 
                                if x.lower() != 'zapytaj o cenę' else np.NaN)

In [ ]:
df.price.head()

In [ ]:
len(df.price.unique())

In [ ]:
len(df[df.price.isna()])

#### Price per m2

In [ ]:
df.price_per_m2.head()

<b>8 309 zł/m²</b> <--- tak wygląda surowa wartość
<br>
<b>8309.0</b> <--- docelowa wartość

In [7]:
df['price_per_m2'] = df['price_per_m2'].apply(lambda x: pd.to_numeric(x.split('z')[0].strip().replace(' ','')) 
                                              if pd.notna(x) else np.NaN)

In [ ]:
df.price_per_m2.head()

In [ ]:
len(df[df.price_per_m2.isna()])

#### Floor

In [ ]:
df.floor.head()

In [ ]:
df.floor.unique()

In [ ]:
df2 = df[['floor_number','floor']].copy()
df2.floor = df2.floor.apply(lambda x: x.split(':')[1].strip() if pd.notnull(x) else np.NaN)
df2.floor_number = df.floor_number.apply(lambda x: x.split(':')[1].strip() if pd.notnull(x) else np.NaN)
parter = df2[df2['floor'] == 'parter'].copy()
parter.floor_number = parter.floor_number.apply(lambda x: pd.to_numeric(x))
# średnia wartość liczby pięter w budnyku, który posiada mieszkanie na poddaszu w kolumnie floor
print("Mean of floor_number column rounded up and having 'poddasze' in floor value: ", round(parter.floor_number.mean()))
# parter[parter.floor == 'Piętro: > 10']
df2.floor.unique()
more_than_10 = df2[df2['floor'] == '> 10'].copy()
# more_then_10.floor.apply(lambda x: pd.to_numeric(x.split(' ')[1]))
more_than_10.floor_number = more_than_10.floor_number.apply(lambda x: pd.to_numeric(x))

In [ ]:
more_than_10.floor_number.median()

In [ ]:
print("Median of floor_number column rounded up and having '> 10' in floor value: ", round(more_than_10.floor_number.median()))

<b>Piętro: 3</b> <--- tak wygląda surowa wartość
<br>
<b>3.0</b> <--- docelowa wartość
<br>
Dodatkowo pojawia się wartości pięter takie jak: 
<ul>
    <li>
        <b>parter</b> - w przypadku takiej wartości wstawiam <i>0</i> ponieważ w polsce liczone jest to jako zerowe piętro
    </li>
    <li>
        <b>suterena</b> - w przypadku takiej wartości wstawiam <i>-1</i> ponieważ jest to mieszkanie poniżej poziomu ulicy, czasami w połowie wystające ponad
    </li>
    <li><b>poddasze</b> - w przypadku takiej wartości wstawiam <i>4</i> ponieważ postanowiłem wstawić w ty miejscu  wartość uśrednioną i zaokrągloną, liczby pięter we wszystkich wierszach posiadających wartość "poddasze" w kolumnie floor</li>
    <li>
        <b> 10</b> - w przypadku takiej wartości wstawiam <i>17</i> ponieważ jest to mediana wszystkich wierszy z kolumny floor_number (liczba pięter) gdzie floor = > 10 
        
        ALBO DAM NAN żeby algorytm wrzucił
    </li>
</ul>

In [8]:
df['floor'] = df['floor'].apply(lambda x: x if x != 'Piętro: > 10' else np.NaN)

In [9]:
df['floor'] = df['floor'].apply(lambda x: pd.to_numeric(x.split(':')[1].strip().replace('parter','0').replace('suterena','-1').replace('poddasze','4')) 
                                if pd.notnull(x) else np.NaN)

In [ ]:
len(df[df.floor.isna()])

In [ ]:
df.floor.unique()

In [ ]:
df.floor.head()

#### Floor number

In [ ]:
df.floor_number.head()

In [ ]:
df.floor_number.unique()

<b>Liczba pięter: 6</b> <--- tak wygląda surowa wartość
<br>
<b>6.0</b> <--- docelowa wartość
<br>
Dodatkowo pojawia się wartoś piętra taka jak: 
<ul>
    <li>
        <b>100</b> - w przypadku takiej wartości wstawiam <i>4</i> ponieważ jest to średnia wszystkich pięter
    </li>
</ul>

In [10]:
df['floor_number'] = df['floor_number'].apply(lambda x: pd.to_numeric(x.split(':')[1].strip()) if pd.notnull(x) else np.NaN)

In [ ]:
df.floor_number.head()

In [ ]:
df.floor_number.unique()

In [ ]:
df.floor_number.describe()

In [ ]:
df.floor_number.median()

In [11]:
df['floor_number'] = df['floor_number'].apply(lambda x: np.NaN if x == 100 else x)

In [12]:
df[df['floor_number'] == 100]

,id,district,price,price_per_m2,floor,floor_number,prod_year,rent,market,building_type,finish_condition,flat_surface_m2,room_number


#### Production Year

In [ ]:
df.prod_year.head()

In [ ]:
df.prod_year.unique()

In [ ]:
print('Rok budowy: 80, ilość przypadków: ',len(df[df.prod_year == "Rok budowy: 80"]))
print('Rok budowy: 90, ilość przypadków: ',len(df[df.prod_year == "Rok budowy: 90"]))
print('Rok budowy: 93, ilość przypadków: ',len(df[df.prod_year == "Rok budowy: 93"]))
print('Rok budowy: 95, ilość przypadków: ',len(df[df.prod_year == "Rok budowy: 95"]))
print('Rok budowy: 1, ilość przypadków: ',len(df[df.prod_year == "Rok budowy: 1"]))
print('Rok budowy: 19, ilość przypadków: ',len(df[df.prod_year == "Rok budowy: 19"]))
print('Rok budowy: 202, ilość przypadków: ',len(df[df.prod_year == "Rok budowy: 202"]))
print('Rok budowy: 1196, ilość przypadków: ',len(df[df.prod_year == "Rok budowy: 1196"]))
print('Rok budowy: 20521, ilość przypadków: ',len(df[df.prod_year == "Rok budowy: 20521"]))

<b>Rok budowy: 1930</b> <--- tak wygląda surowa wartość
<br>
<b>1930</b> <--- docelowa wartość
<br>
Dodatkowo pojawia się wartości rok takie jak: 
<ul>
    <li>
        <b>93, 95, 90, 80</b> - założenie jest takie, że w przypadku tych wartości jest to końcówka 19**, dlatego wstawiam <i>19</i> przed każdy z tych liczb
    </li>
    <li>
        <b>1</b> - za tę wartość wstawiam <i>np.NaN</i> (3 obiekty)
    </li>
    <li>
        <b>20521</b> - założenie, że miał być roku <i>2021</i> (rynek pierwotny)
    </li>
    <li>
        <b>19</b> - założenie, że miał być roku <i>2019</i>
    </li>
    <li>
        <b>202</b> - założenie, że miał być roku <i>2020</i> (rynek pierwotny)
    </li>
    <li>
        <b>1196</b> - za tę wartość wstawiam <i>np.NaN</i>
    </li>
</ul>

In [13]:
df['prod_year'] = df['prod_year'].apply(lambda x: pd.to_numeric(x.split(':')[1].strip()) if pd.notnull(x) else np.NaN)

In [14]:
df['prod_year'] = df['prod_year'].apply(lambda x: x+1900 if x == 93 or x == 95 or x == 90 or x == 80 else x)

In [15]:
df['prod_year'] = df['prod_year'].apply(lambda x: np.NaN if x == 1 else x)

In [16]:
df['prod_year'] = df['prod_year'].apply(lambda x: x+2000 if x == 19 else x)

In [17]:
df['prod_year'] = df['prod_year'].apply(lambda x: x+1818 if x == 202 else x)

In [18]:
df['prod_year'] = df['prod_year'].apply(lambda x: np.NaN if x == 1196 else x)

In [19]:
df['prod_year'] = df['prod_year'].apply(lambda x: x-18500 if x == 20521 else x)

In [20]:
df[df.prod_year >= 2023]

,id,district,price,price_per_m2,floor,floor_number,prod_year,rent,market,building_type,finish_condition,flat_surface_m2,room_number
1136,id48Fdc,Fabryczna,453177.0,6450.0,2.0,3.0,2031.0,Czynsz: 500 zł,Rynek: pierwotny,Rodzaj zabudowy: blok,Stan wykończenia: do wykończenia,"70,26 m²",4 pokoje
2218,id48dOi,Krzyki,667275.0,7750.0,0.0,3.0,2023.0,Czynsz: 750 zł,Rynek: pierwotny,Rodzaj zabudowy: blok,Stan wykończenia: do wykończenia,"86,10 m²",5 pokoi
7048,id48dSZ,Krzyki,448560.0,7200.0,1.0,2.0,2023.0,Czynsz: 500 zł,Rynek: pierwotny,Rodzaj zabudowy: blok,Stan wykończenia: do wykończenia,"62,30 m²",3 pokoje
8174,id48dT1,Krzyki,426600.0,7900.0,0.0,2.0,2023.0,Czynsz: 450 zł,Rynek: pierwotny,Rodzaj zabudowy: blok,Stan wykończenia: do wykończenia,54 m²,3 pokoje


In [21]:
df['prod_year'] = df['prod_year'].apply(lambda x: x - 10 if x == 2031 else x)

In [ ]:
df.prod_year.unique()

#### Rent

In [ ]:
df.rent.head()

In [ ]:
(df.rent.unique())

<b>Czynsz: 375 zł</b> <--- tak wygląda surowa wartość
<br>
<b>375.0</b> <--- docelowa wartość
<br>
Dodatkowo pojawia się wartości czynszu takie jak: 
<ul>
    <li>
        <b>9, 8, 1, 6, 7.4, 4, 12, 7, 1.1, 10, 0.1, 50, 90, 5.5, 70, 8.5, 3, 30, 2, 37, 42</b> - założenie jest takie, że w przypadku tych wartości wstawiam <i>np.NaN</i> ponieważ jest to oczywiście nie prawdziwe
    </li>
</ul>

In [22]:
df['rent'] = df['rent'].apply(lambda x: pd.to_numeric(x.split(':')[1].split('z')[0].replace(' ','').replace(',','.').strip()) 
                              if pd.notnull(x) else np.NaN)

In [23]:
df.rent = df.rent.apply(lambda x: np.NaN if x in [9, 8, 1, 6, 7.4, 4, 12, 7, 1.1, 10, 0.1, 50, 90, 5.5, 70, 8.5, 3, 30, 
                                                  2, 37, 42 ] else x)

In [ ]:
df.rent.unique()

In [ ]:
df.rent.head()

#### Market

In [ ]:
df.market.head()

<b>Rynek: pierwotny</b> <--- tak wygląda surowa wartość
<br>
<b>pierwotny</b> <--- docelowa wartość

In [24]:
df['market'] = df['market'].apply(lambda x: x.split(':')[1].strip() if pd.notnull(x) else np.NaN)

In [ ]:
df.market.head()

In [ ]:
df.market.unique()

In [ ]:
df.market.isnull().sum()

#### Building Type

In [ ]:
df.building_type.head()

<b>Rodzaj zabudowy: kamienica</b> <--- tak wygląda surowa wartość
<br>
<b>kamienica</b> <--- docelowa wartość

In [ ]:
df.building_type.unique()

In [25]:
df['building_type'] = df['building_type'].apply(lambda x: x.split(':')[1].strip() if pd.notnull(x) else np.NaN)

In [ ]:
df.building_type.unique()

#### Finish Condition

In [ ]:
df.finish_condition.head()

In [ ]:
df.finish_condition.unique()

<b>Stan wykończenia: do zamieszkania</b> <--- tak wygląda surowa wartość
<br>
<b>do zamieszkania</b> <--- docelowa wartość

In [26]:
df['finish_condition'] = df['finish_condition'].apply(lambda x: x.split(':')[1].strip() if pd.notnull(x) else np.NaN)

In [ ]:
df.finish_condition.head()

In [ ]:
df.finish_condition.unique()

In [ ]:
df.finish_condition.isnull().sum()

#### Flat Surface m2

In [ ]:
df.flat_surface_m2.head()

In [ ]:
df.flat_surface_m2.unique()

<b>71,01 m²</b> <--- tak wygląda surowa wartość
<br>
<b>71.01</b> <--- docelowa wartość

In [27]:
df['flat_surface_m2'] = df['flat_surface_m2'].apply(lambda x: pd.to_numeric(x.split('m')[0].replace(',','.').strip()) 
                                                    if pd.notnull(x) else np.NaN)

In [ ]:
df.flat_surface_m2.head()

In [ ]:
df.flat_surface_m2.unique()

#### Room Number

In [ ]:
df.room_number.head()

<b>6 pokoi</b> <--- tak wygląda surowa wartość
<br>
<b>6</b> <--- docelowa wartość
<br>
Dodatkowo pojawia się wartości w liczbie pokoii takie jak: 
<ul>
    <li>
        <b>>10</b> - większa ilość pokoii niż 10 brak sprecyzowania 
    </li>
</ul>

In [ ]:
df.room_number.unique()

In [28]:
df[df['room_number'] == '>10 pokoi']

,id,district,price,price_per_m2,floor,floor_number,prod_year,rent,market,building_type,finish_condition,flat_surface_m2,room_number
127,id48CSc,Krzyki,1589500.0,15432.0,NaN,47.0,2013.0,1600.0,wtórny,apartamentowiec,do wykończenia,103.0,>10 pokoi
7424,id48WFo,Fabryczna,440000.0,7333.0,1.0,2.0,NaN,NaN,wtórny,NaN,NaN,60.0,>10 pokoi


In [29]:
df['room_number'] = df['room_number'].apply(lambda x: np.NaN if x == '>10 pokoi' else x)

In [32]:
df.room_number.isna().sum()

2

In [30]:
df['room_number'] = df['room_number'].apply(lambda x: pd.to_numeric(x.split(' ')[0].strip().replace('>10','6')) if pd.notnull(x)  else np.NaN)

In [ ]:
df.room_number.head()

In [ ]:
df.room_number.unique()

In [33]:
df.isna().sum().sort_values(ascending=False)

rent                10266
finish_condition    8553 
building_type       5237 
prod_year           2512 
floor_number        2455 
district            704  
floor               456  
price               297  
price_per_m2        296  
room_number         2    
flat_surface_m2     0    
market              0    
id                  0    
dtype: int64

In [37]:
missing(df)

rent                68.28
finish_condition    56.88
building_type       34.83
prod_year           16.71
floor_number        16.33
district            4.68 
floor               3.03 
price               1.98 
price_per_m2        1.97 
room_number         0.01 
flat_surface_m2     0.00 
market              0.00 
id                  0.00 
dtype: float64


In [ ]:
df.dtypes

In [ ]:
df.describe()

In [ ]:
df.head(10)

In [31]:
df.to_csv('data_set_after_cleaning_with_NaN.csv',index=False)

# Odczytanie oczyszczonego pliku wyłącznie z danymi i np.NaN

In [38]:
df = pd.read_csv('data_set_after_cleaning_with_NaN.csv')

In [42]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15036 entries, 0 to 15035
Data columns (total 13 columns):
id                  15036 non-null object
district            14332 non-null object
price               14739 non-null float64
price_per_m2        14740 non-null float64
floor               14580 non-null float64
floor_number        12581 non-null float64
prod_year           12524 non-null float64
rent                4770 non-null float64
market              15036 non-null object
building_type       9797 non-null object
finish_condition    6483 non-null object
flat_surface_m2     15036 non-null float64
room_number         15034 non-null float64
dtypes: float64(8), object(5)
memory usage: 1.5+ MB


In [43]:
df.drop(columns=['id','rent', 'finish_condition'], inplace=True)

In [44]:
df = df[['district', 'building_type', 'market', 'room_number', 'floor_number', 'floor', 'prod_year', 'flat_surface_m2', 
             'price_per_m2', 'price']]

In [45]:
# df2 = df.query('market != "wtórny"').copy()
df = df[df['market'] != 'wtórny'].copy()

In [46]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9358 entries, 2 to 15035
Data columns (total 10 columns):
district           8998 non-null object
building_type      5019 non-null object
market             9358 non-null object
room_number        9358 non-null float64
floor_number       7243 non-null float64
floor              9086 non-null float64
prod_year          7994 non-null float64
flat_surface_m2    9358 non-null float64
price_per_m2       9064 non-null float64
price              9063 non-null float64
dtypes: float64(7), object(3)
memory usage: 804.2+ KB


tablica numerów indeksów które mają puste pole price

In [52]:
df.isna().sum()

district           360 
building_type      4339
market             0   
room_number        0   
floor_number       2115
floor              272 
prod_year          1364
flat_surface_m2    0   
price_per_m2       294 
price              295 
dtype: int64

In [63]:
def save_nan_index(column_name):
    index = df[column_name].index[df[column_name].apply(np.isnan)]
    df_index = df.index.values.tolist()
    index_list = [df_index.index(i) for i in index]
    return index_list

In [65]:
price_nan_index = save_nan_index('price')
price_per_m2_nan_index = save_nan_index('price_per_m2')
prod_year_nan_index = save_nan_index('prod_year')
floor_nan_index = save_nan_index('floor')
floor_number_nan_index = save_nan_index('floor_number')
# CATEGORICAL NO NEEDED BECAUSE I USED OTHER IMPUTE METHOD
# building_type_nan_index = save_nan_index('building_type')
# district_nan_index = save_nan_index('district')

In [68]:
df.head()

,district,building_type,market,room_number,floor_number,floor,prod_year,flat_surface_m2,price_per_m2,price
2,Stare Miasto,apartamentowiec,pierwotny,2.0,7.0,4.0,2022.0,47.85,9592.0,459000.0
3,Stare Miasto,apartamentowiec,pierwotny,4.0,9.0,8.0,2016.0,212.10,35361.0,7500000.0
5,Krzyki,apartamentowiec,pierwotny,5.0,48.0,NaN,2012.0,227.00,23106.0,5245000.0
6,Krzyki,apartamentowiec,pierwotny,4.0,48.0,NaN,2012.0,176.50,27195.0,4800000.0
9,Stare Miasto,apartamentowiec,pierwotny,6.0,10.0,9.0,2021.0,179.14,20000.0,3582800.0


In [69]:
from missingpy import MissForest
imputer = MissForest(random_state=100)
df_imputed = df.copy()
df_imputed = df_imputed.drop(columns=['district','building_type','market'])
df_imputed = imputer.fit_transform(df_imputed)

Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5
Iteration: 6


In [70]:
df_imputed = pd.DataFrame(data=df_imputed,columns=['room_number','floor_number','floor','prod_year','flat_surface_m2','price_per_m2','price'])
df_imputed = df_imputed.round(decimals=2)
df_imputed.floor_number = df_imputed.floor_number.round(decimals=0)
df_imputed.floor = df_imputed.floor.round(decimals=0)
df_imputed.prod_year = df_imputed.prod_year.round(decimals=0)

In [72]:
df_imputed.iloc[sorted([*prod_year_nan_index, *price_nan_index, *price_per_m2_nan_index, *floor_nan_index, *floor_number_nan_index])]

,room_number,floor_number,floor,prod_year,flat_surface_m2,price_per_m2,price
2,5.0,48.0,8.0,2012.0,227.00,23106.0,5245000.0
3,4.0,48.0,9.0,2012.0,176.50,27195.0,4800000.0
7,5.0,51.0,8.0,2012.0,183.80,18904.0,3474555.0
9,6.0,9.0,8.0,2021.0,179.33,19000.0,3407270.0
11,6.0,10.0,8.0,2021.0,180.00,18929.0,3407270.0
...,...,...,...,...,...,...,...
9349,3.0,4.0,1.0,2021.0,55.19,6570.0,362598.0
9350,2.0,4.0,4.0,2020.0,47.70,7600.0,362520.0
9352,2.0,4.0,3.0,2020.0,49.30,7350.0,362355.0
9354,3.0,5.0,5.0,2021.0,47.80,7575.0,362063.0


## Wprowadzanie categorycznych danych

In [73]:
df_imputed['district'] = df['district'].copy()

In [74]:
df_imputed['building_type'] = df['building_type'].copy()

In [75]:
df_imputed.district.mode()

0    Krzyki
dtype: object

In [76]:
df_imputed.district.fillna('Krzyki', inplace=True)

In [77]:
df_imputed.building_type.mode()

0    blok
dtype: object

In [78]:
df_imputed.building_type.fillna('blok', inplace=True)

In [79]:
df_imputed

,room_number,floor_number,floor,prod_year,flat_surface_m2,price_per_m2,price,district,building_type
0,2.0,7.0,4.0,2022.0,47.85,9592.0,459000.0,Krzyki,blok
1,4.0,9.0,8.0,2016.0,212.10,35361.0,7500000.0,Krzyki,blok
2,5.0,48.0,8.0,2012.0,227.00,23106.0,5245000.0,Stare Miasto,apartamentowiec
3,4.0,48.0,9.0,2012.0,176.50,27195.0,4800000.0,Stare Miasto,apartamentowiec
4,6.0,10.0,9.0,2021.0,179.14,20000.0,3582800.0,Krzyki,blok
...,...,...,...,...,...,...,...,...,...
9353,2.0,4.0,1.0,2020.0,51.03,7100.0,362313.0,Krzyki,blok
9354,3.0,5.0,5.0,2021.0,47.80,7575.0,362063.0,Krzyki,blok
9355,2.0,4.0,1.0,2020.0,51.03,7094.0,362000.0,Krzyki,blok
9356,3.0,2.0,0.0,2021.0,52.20,6933.0,361900.0,Psie Pole,blok


In [80]:
df_imputed.floor = df_imputed.floor.astype('int64')
df_imputed.floor_number = df_imputed.floor_number.astype('int64')
df_imputed.prod_year = df_imputed.prod_year.astype('int64')
df_imputed.room_number = df_imputed.room_number.astype('int64')

In [81]:
df_imputed.describe().round(decimals=2)

,room_number,floor_number,floor,prod_year,flat_surface_m2,price_per_m2,price
count,9358.00,9358.00,9358.00,9358.00,9358.00,9358.00,9358.00
mean,2.75,4.37,2.16,2020.16,58.67,8724.95,507907.30
std,0.96,2.27,1.79,5.23,21.83,2077.47,270827.66
min,1.00,1.00,0.00,1894.00,16.97,3840.00,128944.00
25%,2.00,3.00,1.00,2020.00,44.90,7220.50,379000.00
50%,3.00,4.00,2.00,2020.00,54.00,8425.00,449000.00
75%,3.00,5.00,3.00,2021.00,67.87,9759.00,552716.50
max,8.00,51.00,10.00,2023.00,305.82,35361.00,7500000.00


In [87]:
df_imputed.to_csv('flats_wroclaw_final.csv',index=False)

## TEST ALGORYTMU MISSFOREST

In [ ]:
# Generate unique lists of random integers
inds1 = list(set(np.random.randint(0, len(df_nan_drop), 10)))
inds2 = list(set(np.random.randint(0, len(df_nan_drop), 15)))

# Replace the values at given index position with NaNs
# df_nan['price'] = [val if i not in inds1 else np.nan for i, val in enumerate(df_nan['price'])]
df_nan_drop['room_number'] = [val if i not in inds1 else np.nan for i, val in enumerate(df_nan_drop['room_number'])]
df_nan_drop['floor'] = [val if i not in inds2 else np.nan for i, val in enumerate(df_nan_drop['floor'])]
# df_nan['price_per_m2'] = [val if i not in inds2 else np.nan for i, val in enumerate(df_nan['price_per_m2'])]


# Get count of missing values by column
df_nan_drop.isnull().sum()

In [ ]:
df_orygi = df_nan.dropna().copy()

In [ ]:
from missingpy import MissForest
imputer = MissForest(random_state=100)
X = df_nan_drop.drop(columns=['district','building_type','price','price_per_m2','flat_surface_m2','prod_year'],axis=1)
# cat_cols = [df_orygi.columns.get_loc(col) for col in df_orygi.select_dtypes(['category']).columns.tolist()]
# cat_cols
# X.dtypes
X_imputed = imputer.fit_transform(X)

In [ ]:
# df_nan['MF_price'] = X_imputed[:, 0]
df_orygi['MF_room_number'] = X_imputed[:, 0]
df_orygi['MF_floor'] = X_imputed[:, 2]
# df_nan['MF_price_per_m2'] = X_imputed[:, 6]
# comparison_df = df_nan[['price', 'MF_price', 'room_number', 'MF_room_number', 'floor','MF_floor','price_per_m2','MF_price_per_m2']]
comparison_df = df_orygi[['room_number', 'MF_room_number', 'floor','MF_floor']]
# comparison_df['ABS_ERROR_price'] = np.abs(comparison_df['price'] - comparison_df['MF_price'])
comparison_df['ABS_ERROR_room_number'] = np.abs(comparison_df['room_number'] - comparison_df['MF_room_number'])
comparison_df['ABS_ERROR_floor'] = np.abs(comparison_df['floor'] - comparison_df['MF_floor'])
# comparison_df['ABS_ERROR_price_per_m2'] = np.abs(comparison_df['price_per_m2'] - comparison_df['MF_price_per_m2'])
comparison_df.round(decimals=2)
comparison_df.iloc[sorted([*inds1, *inds2])].head(59)

In [ ]:
columns = list(df_nan.columns)
columns.remove('district')
columns.remove('building_type')

In [ ]:
df_imputed = pd.DataFrame(data=Xtransform,columns=columns)

In [ ]:
df_imputed.round(decimals=2)

In [ ]:
df_orygi.district = df_orygi.district.astype('category')
df_orygi.building_type = df_orygi.building_type.astype('category')

#### This option displays full DataFrame

In [ ]:
pd.set_option('display.max_rows', df.shape[0]+1)